In [ ]:
import pandas as pd

# Nur die ersten 5 Zeilen einlesen
df_preview = pd.read_csv(r'C:\Users\kaiws\Downloads\311_Service_Requests_from_2010_to_Present_20250621.csv', nrows=5)

# Spaltennamen anzeigen
print("Spaltennamen:")
print(df_preview.columns)
print("\n" + "="*50 + "\n")

# Erste 5 Zeilen anzeigen
print("Erste 5 Zeilen:")
print(df_preview)

In [ ]:
import pandas as pd
from datetime import datetime

# Gewünschte Spalten
selected_columns = ['Created Date', 'Closed Date', 'Agency', 'Complaint Type', 
                   'Descriptor', 'Location Type', 'Incident Zip', 'Borough','Vehicle Type','Location']

# Dateipfad
csv_path = r'C:\Users\kaiws\Downloads\311_Service_Requests_from_2010_to_Present_20250621.csv'

# Liste für gefilterte Daten
filtered_data = []

# Zähler für Gesamtstatistik
total_rows = 0
filtered_rows = 0

# Chunkweise einlesen und filtern
chunk_size = 100000  # 100k Zeilen pro Chunk

for chunk in pd.read_csv(csv_path, 
                        usecols=selected_columns, 
                        chunksize=chunk_size,
                        low_memory=False):
    
    # Datums-Spalten konvertieren mit explizitem Format
    chunk['Created Date'] = pd.to_datetime(chunk['Created Date'], 
                                         format='%m/%d/%Y %I:%M:%S %p', 
                                         errors='coerce')
    chunk['Closed Date'] = pd.to_datetime(chunk['Closed Date'], 
                                        format='%m/%d/%Y %I:%M:%S %p', 
                                        errors='coerce')
    
    # Gesamtanzahl aktualisieren
    total_rows += len(chunk)
    
    # Filter: Nur Jahre 2022, 2023, 2024
    year_filter = chunk['Created Date'].dt.year.isin([2022, 2023, 2024])
    
    # Filter: Closed Date darf nicht leer sein
    closed_date_filter = chunk['Closed Date'].notna()
    
    # Kombinierte Filter anwenden
    filtered_chunk = chunk[year_filter & closed_date_filter]
    
    # Gefilterte Anzahl aktualisieren
    filtered_rows += len(filtered_chunk)
    
    # Gefilterte Daten zur Liste hinzufügen
    filtered_data.append(filtered_chunk)
    
    # Fortschritt anzeigen
    print(f"Chunk verarbeitet: {len(chunk)} Zeilen gelesen, {len(filtered_chunk)} Zeilen gefiltert")

# Alle gefilterten Chunks zusammenführen
df_filtered = pd.concat(filtered_data, ignore_index=True)

# Gefilterte Daten speichern
df_filtered.to_csv(r'C:\Users\kaiws\Downloads\filtered_311_data_2022-2024_pycaret_new.csv', index=False))

print(f"\n{'='*60}")
print(f"GESAMTSTATISTIK:")
print(f"{'='*60}")
print(f"Gesamtanzahl Datensätze in der CSV: {total_rows:,}")
print(f"Gefilterte Datensätze (2022-2024, mit Closed Date): {filtered_rows:,}")
print(f"Entfernte Datensätze: {total_rows - filtered_rows:,}")
print(f"Filterrate: {((total_rows - filtered_rows) / total_rows * 100):.1f}% der Daten entfernt")
print(f"{'='*60}")

print(f"\nFinale Datenmenge: {len(df_filtered)} Zeilen")
print(f"Spalten: {list(df_filtered.columns)}")

# Erste Zeilen anzeigen
print("\nErste 5 Zeilen:")
print(df_filtered.head())

# Jahresverteilung anzeigen
print("\nJahresverteilung:")
print(df_filtered['Created Date'].dt.year.value_counts().sort_index())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Korrekter Pfad zu den gefilterten Daten
csv_path = r'C:\Users\kaiws\Downloads\filtered_311_data_2022-2024_pycaret_new.csv'

# Daten laden (mit low_memory=False um Warnung zu vermeiden)
df_filtered = pd.read_csv(csv_path, low_memory=False)
print("Daten erfolgreich geladen!")

# Analyse der Complaint Types
print("ANALYSE DER COMPLAINT TYPES")
print("="*50)

# Gesamtzahl der unterschiedlichen Complaint Types
total_unique_complaints = df_filtered['Complaint Type'].nunique()
print(f"Gesamtzahl unterschiedlicher Complaint Types: {total_unique_complaints:,}")

# Häufigste 30 Complaint Types
top_30_complaints = df_filtered['Complaint Type'].value_counts().head(30)

print(f"\nDie 30 häufigsten Complaint Types:")
print("-" * 50)

# Mit Index (Rang) und Prozentangabe
for i, (complaint_type, count) in enumerate(top_30_complaints.items(), 1):
    percentage = (count / len(df_filtered)) * 100
    print(f"{i:2d}. {complaint_type:<40} {count:>8,} ({percentage:5.1f}%)")

# Zusätzliche Statistiken
print(f"\n{'='*50}")
print("ZUSÄTZLICHE STATISTIKEN:")
print(f"Gesamtzahl Datensätze: {len(df_filtered):,}")
print(f"Durchschnittliche Häufigkeit pro Complaint Type: {len(df_filtered) / total_unique_complaints:,.0f}")
print(f"Complaint Types mit nur 1 Eintrag: {(df_filtered['Complaint Type'].value_counts() == 1).sum()}")
print(f"Complaint Types mit ≤ 10 Einträgen: {(df_filtered['Complaint Type'].value_counts() <= 10).sum()}")

# VISUALISIERUNG
print(f"\n{'='*50}")
print("VISUALISIERUNG:")
print("="*50)

# Top 10 Complaint Types als Balkendiagramm
plt.figure(figsize=(14, 10))

# Daten für Visualisierung vorbereiten
top_10_complaints = df_filtered['Complaint Type'].value_counts().head(10)

# Balkendiagramm erstellen (KORRIGIERT - ohne Warnung)
sns.barplot(x=top_10_complaints.values, y=top_10_complaints.index, color='steelblue')

plt.title('Top 10 Complaint Types (2022-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Anzahl der Service Requests', fontsize=12)
plt.ylabel('Complaint Type', fontsize=12)

# Werte auf den Balken anzeigen
for i, v in enumerate(top_10_complaints.values):
    plt.text(v + max(top_10_complaints.values) * 0.01, i, f'{v:,}', 
             va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Zusätzlich: Verteilung als Tortendiagramm (Top 5)
plt.figure(figsize=(10, 8))
top_5_complaints = df_filtered['Complaint Type'].value_counts().head(5)
other_complaints = df_filtered['Complaint Type'].value_counts().iloc[5:].sum()

# Daten für Tortendiagramm vorbereiten
pie_data = pd.concat([top_5_complaints, pd.Series({'Sonstige': other_complaints})])

plt.pie(pie_data.values, labels=pie_data.index, autopct='%1.1f%%', startangle=90)
plt.title('Verteilung der Complaint Types (Top 5 vs. Sonstige)', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.show()


In [ ]:
# Prüfe ob df_filtered existiert
if 'df_filtered' in locals():
    print("df_filtered existiert und wird verwendet.")
    print(f"Anzahl Zeilen: {len(df_filtered):,}")
else:
    print("df_filtered existiert nicht. Lade Daten neu...")
    # Lade Daten neu
    csv_path = r'C:\Users\kaiws\Downloads\filtered_311_data_2022-2024_pycaret_new.csv'
    df_filtered = pd.read_csv(csv_path, low_memory=False)
    print("Daten geladen!")

# Top 30 Complaint Types identifizieren
top_30_complaint_types = df_filtered['Complaint Type'].value_counts().head(30).index

print(f"Anzahl Complaint Types vor Filterung: {df_filtered['Complaint Type'].nunique():,}")
print(f"Anzahl Datensätze vor Filterung: {len(df_filtered):,}")

# Nur Zeilen mit den Top 30 Complaint Types behalten
df_filtered_top30 = df_filtered[df_filtered['Complaint Type'].isin(top_30_complaint_types)].copy()

print(f"\nAnzahl Complaint Types nach Filterung: {df_filtered_top30['Complaint Type'].nunique():,}")
print(f"Anzahl Datensätze nach Filterung: {len(df_filtered_top30):,}")
print(f"Entfernte Datensätze: {len(df_filtered) - len(df_filtered_top30):,}")
print(f"Behaltene Daten: {((len(df_filtered_top30) / len(df_filtered)) * 100):.1f}%")

# Überprüfung: Zeige die Top 30 Complaint Types
print(f"\nDie 30 häufigsten Complaint Types (die behalten wurden):")
print("-" * 60)
for i, complaint_type in enumerate(top_30_complaint_types, 1):
    count = df_filtered_top30[df_filtered_top30['Complaint Type'] == complaint_type].shape[0]
    print(f"{i:2d}. {complaint_type}")

# Gefilterte Daten speichern
output_path = r'C:\Users\kaiws\Downloads\filtered_311_data_top30.csv'
df_filtered_top30.to_csv(output_path, index=False)
print(f"\nGefilterte Daten gespeichert in: {output_path}")

# Aktualisiere df_filtered für weitere Analysen
df_filtered = df_filtered_top30.copy()
print("\nDataFrame 'df_filtered' wurde mit den Top 30 Complaint Types aktualisiert!")